# Тестирование модели разрешения кореференции SpanBERT+BERT-Chinese 

In [0]:
%cd coref

/coref/coref


In [0]:
!cat requirements.txt | sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/' > tmp
!mv tmp requirements.txt

In [0]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install -r requirements.txt --log install-log.txt -q

Установка BERTCoref

In [0]:
! ./setup_all.sh

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
import os
import json
import warnings
warnings.filterwarnings('ignore')

In [0]:
%cd ..

/coref


Загрузка модели


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp '/content/drive/My Drive/colab_data/Anaphora/train_bert_base.zip' ./

In [0]:
!unzip '/content/drive/My Drive/colab_data/Anaphora/train_bert_base.zip'

Archive:  /content/drive/My Drive/colab_data/Anaphora/train_bert_base.zip
replace train_bert_base/bert_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
!mkdir /content/bert_base

mkdir: cannot create directory ‘/content/bert_base’: File exists


In [0]:
!cp -r /content/train_bert_base/* /content/bert_base

В ячейку ниже добавляются различные сообщения с китайского медицинского форума [haodf.com](https://www.haodf.com)

In [0]:
text = [
'X光检查三颗智齿歪斜，其中一颗已经长出来，四月X光检查出有.',
'脸上长痣比较多，感觉越来越多，想把它们点掉!',
'粉瘤，其中一个已结疤，想做了去，用住院么有两个粉瘤，好...',
]

Настройки: жанр текста и название модели. 

In [0]:
genre = "wb"
# The Ontonotes data for training the model contains text from several sources
# of very different styles. You need to specify the most suitable one out of:
# "bc": broadcast conversation
# "bn": broadcast news
# "mz": magazine
# "nw": newswire
# "pt": Bible text
# "tc": telephone conversation
# "wb": web data
model_name = 'bert_base'

In [0]:
os.environ['data_dir'] = '/content/'
os.environ['CHOSEN_MODEL'] = model_name

Подготовка данных перед передачей модели.

In [0]:
from coref.bert import tokenization

In [0]:
input_filename = 'sample.in.json'
output_filename = 'sample.out.txt'

In [0]:
data = {
    'doc_key': genre,
    'sentences': [["[CLS]"]],
    'speakers': [["[SPL]"]],
    'clusters': [],
    'sentence_map': [0],
    'subtoken_map': [0],
}

In [0]:
# Максимальная длина сегмента
max_segment = 512

In [0]:
tokenizer = tokenization.FullTokenizer(vocab_file='/content/train_bert_base/vocab.txt', do_lower_case=False)

In [0]:
subtoken_num = 0
for sent_num, line in enumerate(text):
    raw_tokens = line.split()
    tokens = tokenizer.tokenize(line)
    if len(tokens) + len(data['sentences'][-1]) >= max_segment:
        data['sentences'][-1].append("[SEP]")
        data['sentences'].append(["[CLS]"])
        data['speakers'][-1].append("[SPL]")
        data['speakers'].append(["[SPL]"])
        data['sentence_map'].append(sent_num - 1)
        data['subtoken_map'].append(subtoken_num - 1)
        data['sentence_map'].append(sent_num)
        data['subtoken_map'].append(subtoken_num)

    ctoken = raw_tokens[0]
    cpos = 0
    for token in tokens:
        data['sentences'][-1].append(token)
        data['speakers'][-1].append("-")
        data['sentence_map'].append(sent_num)
        data['subtoken_map'].append(subtoken_num)
        
        if token.startswith("##"):
            token = token[2:]
        if len(ctoken) == len(token):
            subtoken_num += 1
            cpos += 1
            if cpos < len(raw_tokens):
                ctoken = raw_tokens[cpos]
        else:
            ctoken = ctoken[len(token):]

data['sentences'][-1].append("[SEP]")
data['speakers'][-1].append("[SPL]")
data['sentence_map'].append(sent_num - 1)
data['subtoken_map'].append(subtoken_num - 1)

with open("sample.in.json", 'w') as out:
    json.dump(data, out, sort_keys=True)

In [0]:
! cat sample.in.json

{"clusters": [], "doc_key": "wb", "sentence_map": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1], "sentences": [["[CLS]", "[UNK]", "\u5149", "\u68c0", "\u67e5", "\u4e09", "\u9897", "\u667a", "\u9f7f", "\u6b6a", "\u659c", "\uff0c", "\u5176", "\u4e2d", "\u4e00", "\u9897", "\u5df2", "\u7ecf", "\u957f", "\u51fa", "\u6765", "\uff0c", "\u56db", "\u6708", "[UNK]", "\u5149", "\u68c0", "\u67e5", "\u51fa", "\u6709", ".", "\u8138", "\u4e0a", "\u957f", "\u75e3", "\u6bd4", "\u8f83", "\u591a", "\uff0c", "\u611f", "\u89c9", "\u8d8a", "\u6765", "\u8d8a", "\u591a", "\uff0c", "\u60f3", "\u628a", "\u5b83", "\u4eec", "\u70b9", "\u6389", "!", "\u7c89", "\u7624", "\uff0c", "\u5176", "\u4e2d", "\u4e00", "\u4e2a", "\u5df2", "\u7ed3", "\u75a4", "\uff0c", "\u60f3", "\u505a", "\u4e86", "\u53bb", "\uff0c", "\u7

Заходим обратно в папку скриптов и выполняем предикт

In [0]:
%cd coref

/coref/coref


In [0]:
! GPU=0 python predict.py $CHOSEN_MODEL ../sample.in.json ../sample.out.txt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Выводим полученный результат

In [0]:
%cd ..

/coref


In [0]:
output = json.load(open("sample.out.txt"))

comb_text = [word for sentence in output['sentences'] for word in sentence]

def convert_mention(mention):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

seen = set()
print('Clusters:')
for cluster in output['predicted_clusters']:
    mapped = []
    for mention in cluster:
        seen.add(tuple(mention))
        mapped.append(convert_mention(mention))
    print(mapped, end=",\n")

print('\nMentions:')
for mention in output['top_spans']:
    if tuple(mention) in seen:
        continue
    print(convert_mention(mention), end=",\n")

Clusters:
[((0, 1), '三 颗 智 齿'), ((0, 1), '其 中')],
[((0, 1), '痣'), ((0, 1), '它 们')],
[((1, 2), '粉 瘤'), ((1, 2), '其 中')],

Mentions:
((0, 1), '[UNK]'),
((0, 1), '[UNK] 光'),
((0, 1), '三 颗 智 齿 歪 斜'),
((0, 1), '智 齿'),
((0, 1), '其 中 一 颗'),
((0, 1), '一 颗'),
((0, 1), '四'),
((0, 1), '四 月'),
((0, 1), '四 月 [UNK] 光'),
((0, 1), '[UNK] 光'),
((0, 1), '.'),
((0, 1), '脸'),
((0, 1), '脸 上'),
((0, 1), '感 觉'),
((0, 1), '它 们 点 掉 !'),
((0, 1), '!'),
((1, 2), '其 中 一 个'),
((1, 2), '一 个'),
((1, 2), '疤'),
((1, 2), '去'),
((1, 2), '用'),
((1, 2), '用 住 院 么'),
((1, 2), '住 院'),
((1, 2), '住 院 么'),
((1, 2), '两 个 粉 瘤'),
((1, 2), '两 个 粉 瘤 ， 好 . . .'),
((1, 2), '好 . . .'),
